# Sample 01: OpenAI SDK ద్వారా త్వరిత చాట్

ఈ నోట్‌బుక్ Microsoft Foundry Local తో OpenAI SDK ను త్వరిత చాట్ ఇంటరాక్షన్ల కోసం ఎలా ఉపయోగించాలో చూపిస్తుంది.

## అవలోకనం

ఈ ఉదాహరణ చూపిస్తుంది:
- Foundry Local తో OpenAI Python SDK ఉపయోగించడం
- Azure OpenAI మరియు స్థానిక Foundry కాన్ఫిగరేషన్లను రెండింటినీ నిర్వహించడం
- సరైన లోప నిర్వహణ మరియు fallback వ్యూహాలను అమలు చేయడం
- సర్వీస్ నిర్వహణ కోసం FoundryLocalManager ఉపయోగించడం


## ముందస్తు అవసరాలు

మీకు ఉన్నదని నిర్ధారించుకోండి:
1. Foundry Local ఇన్‌స్టాల్ చేసి నడుస్తోంది
2. ఒక మోడల్ లోడ్ చేయబడింది (ఉదా: `foundry model run phi-4-mini`)
3. అవసరమైన Python ప్యాకేజీలు ఇన్‌స్టాల్ చేయబడ్డాయి

### డిపెండెన్సీలు ఇన్‌స్టాల్ చేయండి


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## అవసరమైన లైబ్రరీలను దిగుమతి చేసుకోండి


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## కాన్ఫిగరేషన్

మీ కాన్ఫిగరేషన్‌ను సెట్ చేయండి. మీరు ఈ రెండు లో ఏదైనా ఉపయోగించవచ్చు:
1. **Azure OpenAI** (క్లౌడ్-ఆధారిత)
2. **Foundry Local** (లోకల్ ఇన్ఫరెన్స్)

### ఎంపిక 1: Azure OpenAI కాన్ఫిగరేషన్


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### ఎంపిక 2: ఫౌండ్రీ లోకల్ కాన్ఫిగరేషన్


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## క్లయింట్ ప్రారంభం

మీ కాన్ఫిగరేషన్ ఆధారంగా OpenAI క్లయింట్‌ను సృష్టించండి:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## ప్రాథమిక చాట్ ఉదాహరణ

సాధారణ చాట్ పరస్పర చర్యను పరీక్షిద్దాం:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## ఇంటరాక్టివ్ చాట్ సెషన్

మోడల్ ఎలా స్పందిస్తుందో చూడటానికి వివిధ రకాల ప్రాంప్ట్‌లను ప్రయత్నించండి:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## అధునాతన ఉపయోగం: స్ట్రీమింగ్ ప్రతిస్పందనలు

దీర్ఘమైన ప్రతిస్పందనల కోసం, స్ట్రీమింగ్ మెరుగైన వినియోగదారు అనుభవాన్ని అందించవచ్చు:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## లోపం నిర్వహణ మరియు నిర్ధారణ

సేవ స్థితి మరియు అందుబాటులో ఉన్న మోడల్స్‌ను పరిశీలిద్దాం:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## కస్టమ్ ప్రాంప్ట్ పరీక్ష

మీ స్వంత ప్రాంప్ట్‌లను పరీక్షించడానికి క్రింది సెల్‌ను ఉపయోగించండి:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## సారాంశం

ఈ నోట్‌బుక్ చూపించింది:

1. **✅ క్లయింట్ సెటప్**: Foundry Local తో OpenAI SDK ను ఎలా కాన్ఫిగర్ చేయాలో
2. **✅ ప్రాథమిక చాట్**: సాదా అభ్యర్థన-ప్రతిస్పందన పరస్పర చర్యలు
3. **✅ స్ట్రీమింగ్**: రియల్-టైమ్ ప్రతిస్పందన ఉత్పత్తి
4. **✅ లోపాల నిర్వహణ**: బలమైన లోపాల నిర్వహణ మరియు డయాగ్నోస్టిక్స్
5. **✅ సర్వీస్ ఆరోగ్యం**: మోడల్ అందుబాటును మరియు సర్వీస్ స్థితిని తనిఖీ చేయడం

### తదుపరి దశలు

- **సాంపిల్ 02**: Azure OpenAI మద్దతుతో అభివృద్ధి చెందిన SDK సమీకరణ
- **సాంపిల్ 04**: Chainlit చాట్ అప్లికేషన్ నిర్మాణం
- **సాంపిల్ 05**: బహుళ ఏజెంట్ సమన్వయ వ్యవస్థలు
- **సాంపిల్ 06**: తెలివైన మోడల్ రూటింగ్

### Foundry Local యొక్క ముఖ్య లాభాలు

- 🔒 **గోప్యత**: డేటా మీ పరికరం నుండి ఎప్పుడూ బయటకు వెళ్లదు
- ⚡ **వేగం**: తక్కువ ఆలస్యం స్థానిక అంచనా
- 💰 **ఖర్చు**: ఏ API వినియోగ ఖర్చులు లేవు
- 🔌 **ఆఫ్‌లైన్**: ఇంటర్నెట్ కనెక్షన్ లేకుండా పనిచేస్తుంది
- 🛠️ **అనుకూలత**: OpenAI-అనుకూల API


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారుల కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
